In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input



D0814 10:24:05.804849802      16 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0814 10:24:05.804875082      16 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0814 10:24:05.804879237      16 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0814 10:24:05.804882454      16 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0814 10:24:05.804885299      16 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0814 10:24:05.804888352      16 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0814 10:24:05.804891119      16 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0814 10:24:05.

In [2]:
data_paths = ['/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Dyskeratotic/im_Dyskeratotic/CROPPED', '/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Koilocytotic/im_Koilocytotic/CROPPED', '/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Metaplastic/im_Metaplastic/CROPPED', '/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/CROPPED', '/kaggle/input/cervical-cancer-largest-dataset-sipakmed/im_Superficial-Intermediate/im_Superficial-Intermediate/CROPPED']
labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic', 'im_Parabasal', 'im_Superficial-Intermediate']


In [3]:
images = []
image_labels = []


In [4]:
for label, data_path in zip(labels, data_paths):
    for file in os.listdir(data_path):
        if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.bmp'):
            image_path = os.path.join(data_path, file)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)
            images.append(image)
            image_labels.append(label)

In [5]:
train_images, test_images, train_labels, test_labels = train_test_split(images, image_labels, test_size=0.2, random_state=42)


In [6]:
# Print the number of train and test samples
print("Train samples:", len(train_images))
print("Test samples:", len(test_images))


Train samples: 3239
Test samples: 810


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Define the path to the directory where augmented images will be saved
augmented_images_dir = '/kaggle/temp/'

# Create the directory for augmented images if it doesn't exist
os.makedirs(augmented_images_dir, exist_ok=True)

# Convert train_images list to a NumPy array
train_images = np.array(train_images)

# Get the number of images in the original training dataset
num_original_images = train_images.shape[0]
print(num_original_images)

# Define the desired number of augmented images
desired_num_augmented_images = 5000
# Calculate the number of augmentation iterations needed
num_iterations = desired_num_augmented_images // num_original_images

print(num_iterations)

# Define the augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Create lists to store augmented images and labels
augmented_images = []
augmented_labels = []

# Generate augmented images and save them
for i in range(num_iterations):
    # Iterate over the original training images
    for j in range(num_original_images):
        img = train_images[j]
        label = train_labels[j]

        # Expand the dimensions of the image to match the expected input shape of the generator
        img = np.expand_dims(img, axis=0)

        # Generate augmented images
        augment_iter = datagen.flow(img, batch_size=1, save_to_dir=augmented_images_dir, save_prefix='augmented_', save_format='bmp')

        # Generate one augmented image for each original image
        augment_img = next(augment_iter)[0]  # Get the augmented image from the batch
        augment_label = label

        # Add the augmented image and label to the respective lists
        augmented_images.append(augment_img)
        augmented_labels.append(augment_label)
    print(len(augmented_images))

    # Check if the desired number of augmented images is reached
    if len(augmented_images) >= desired_num_augmented_images:
        break

# Convert the augmented images and labels lists to NumPy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Reshape the augmented_images array to match the shape of the original images
augmented_images = np.reshape(augmented_images, (-1,) + train_images.shape[1:])

# Concatenate the original images and labels with the augmented images and labels
train_images = np.concatenate((train_images, augmented_images), axis=0)
train_labels = np.concatenate((train_labels, augmented_labels), axis=0)

# After running this code, your training dataset will be augmented with 5000 additional images.



3239
1
3239


In [8]:
# Print the number of train and test samples
print("Train samples:", len(train_images))
print("Test samples:", len(test_images))


Train samples: 6478
Test samples: 810


In [9]:
test_labels=np.array(test_labels)
test_images=np.array(test_images)
train_images=np.array(train_images)
train_labels=np.array(train_labels)

In [10]:
train_images/=255.0
test_images/=255.0

In [11]:
print(test_images.shape)
print(train_images.shape)
print(train_labels.shape)
print(test_labels.shape)

(810, 224, 224, 3)
(6478, 224, 224, 3)
(6478,)
(810,)


In [12]:
from tensorflow.keras.applications import VGG16, InceptionV3, DenseNet121
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [13]:

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))



58889256/58889256 [==============================] - 0s 0us/step


In [14]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
# Extract features from the desired layers of VGG16 
train_features_vgg = vgg_model.get_layer('block5_pool').output

In [16]:
# Create feature extraction models
from tensorflow.keras import Model

In [17]:
vgg_features_model = Model(inputs=vgg_model.input, outputs=train_features_vgg)


In [18]:
# Extract features from the desired layers of VGG16 for train images
train_features_vgg = vgg_features_model.predict(train_images, batch_size=100)

65/65 [==============================] - 108s 2s/step


In [19]:
# Extract features from the desired layers of VGG16 for test images
test_features_vgg = vgg_features_model.predict(test_images, batch_size=100)


9/9 [==============================] - 14s 1s/step


In [24]:
from sklearn.linear_model import LogisticRegression

In [20]:
print(train_features_vgg.shape)
print(test_features_vgg.shape)

(6478, 7, 7, 512)
(810, 7, 7, 512)


In [21]:
train_features_2d=train_features_vgg.reshape(train_features_vgg.shape[0], -1)
test_features_2d=test_features_vgg.reshape(test_features_vgg.shape[0], -1)

In [22]:
print(train_features_2d.shape)
print(test_features_2d.shape)

(6478, 25088)
(810, 25088)


In [25]:
lr_model_selected = LogisticRegression()
lr_model_selected.fit(train_features_2d, train_labels)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
# Make predictions on the test data
y_pred = lr_model_selected.predict(test_features_2d)

# Calculate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8876543209876543


In [28]:
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


In [29]:
import warnings
warnings.filterwarnings('ignore')

In [70]:
lr_model = LogisticRegression()

#2,5

In [110]:
rfe = RFE(estimator=lr_model, n_features_to_select=2000,step=5000,verbose=1)


In [111]:
rfe.fit(train_features_2d, train_labels)

Fitting estimator with 25088 features.
Fitting estimator with 20088 features.
Fitting estimator with 15088 features.
Fitting estimator with 10088 features.
Fitting estimator with 5088 features.


RFE(estimator=LogisticRegression(), n_features_to_select=2000, step=5000,
    verbose=1)

In [112]:
# Select the corresponding features from the training and test sets
train_features_selected = train_features_2d[:, rfe.support_]
test_features_selected = test_features_2d[:, rfe.support_]

In [113]:
lr_model_selected = LogisticRegression()
lr_model_selected.fit(train_features_selected, train_labels)

LogisticRegression()

In [114]:
# Make predictions on the test data
y_pred = lr_model_selected.predict(test_features_selected)

# Calculate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9049382716049382
